<a href="https://colab.research.google.com/github/venkatreddyt/HW/blob/master/Current_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


TODO: Cleanup text fields 
add overiview of code at the top
2. references



In [1]:
#Import all the necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%tensorflow_version 1.x
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Model, Input, Sequential
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint 
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda, Layer, Concatenate
from keras import backend as K


TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
#Mount google drive (uploaded 'ner_dataset.csv' to google drive)
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#read the file from drive
df = pd.read_csv("/content/gdrive/My Drive/ner_dataset.csv", encoding="latin1")
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [4]:
#we can see missing values for sentence, let get statistics on data
df.isnull().sum()


Sentence #    1000616
Word                0
POS                 0
Tag                 0
dtype: int64

In [5]:
#fill the missing values
df = df.fillna(method="ffill")
df.head(5)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [6]:
#desc on data frame
df.describe()

,Sentence #,Word,POS,Tag
count,1048575,1048575,1048575,1048575
unique,47959,35178,42,17
top,Sentence: 22480,the,NN,O
freq,104,52573,145807,887908


There are 47959 unique sentences, 35178 unique words and 17 Tags.

In [7]:
#dropping POS tag, since its not needed
df = df.drop(['POS'], axis =1)
df.head(5)

,Sentence #,Word,Tag
0,Sentence: 1,Thousands,O
1,Sentence: 1,of,O
2,Sentence: 1,demonstrators,O
3,Sentence: 1,have,O
4,Sentence: 1,marched,O


In [8]:
class SentenceExtract(object):
    
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        aggreate_function = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(aggreate_function)
        self.sentences = [s for s in self.grouped]
    
    def get_txt(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [9]:
get_sent = SentenceExtract(df)
sent_with_tags = get_sent.get_txt()
#sentences with NE tags
print(sent_with_tags)

[('Thousands', 'O'), ('of', 'O'), ('demonstrators', 'O'), ('have', 'O'), ('marched', 'O'), ('through', 'O'), ('London', 'B-geo'), ('to', 'O'), ('protest', 'O'), ('the', 'O'), ('war', 'O'), ('in', 'O'), ('Iraq', 'B-geo'), ('and', 'O'), ('demand', 'O'), ('the', 'O'), ('withdrawal', 'O'), ('of', 'O'), ('British', 'B-gpe'), ('troops', 'O'), ('from', 'O'), ('that', 'O'), ('country', 'O'), ('.', 'O')]


In [10]:
sentences = get_sent.sentences
longest_sentence = max(len(s) for s in sentences)
print(longest_sentence, 'words are present in longest sentence')

104 words are present in longest sentence


In [11]:
#list of unique tags
tags=list(set(df["Tag"].values))
print('list of unique tags:',tags)

list of unique tags: ['O', 'B-art', 'I-geo', 'I-tim', 'I-gpe', 'B-nat', 'B-eve', 'I-eve', 'I-nat', 'I-org', 'I-per', 'B-geo', 'I-art', 'B-gpe', 'B-per', 'B-org', 'B-tim']


In [12]:
#list of unique words
padding = '_PAD_'
words = list(set(df['Word'].values))
words.append(padding)
print(words[0:10])
print('Number of unique words:', len(words))

['Dwain', 'Muluzi', 'Mladen', 'panel', '1980', 'Oxen', 'Polska', 'Lance', 'Levy', 'wanting']
Number of unique words: 35179


Prepare sentences to be of equal size

In [13]:
word_2_index = {w:i for i,w in enumerate(words)}
max_len = 104
x_pad = []
X = [[w[0]for w in s] for s in sentences]
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append(padding)
    x_pad.append(new_seq)

x_pad[0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_',
 '_PAD_'

Pad labels accordingly with O label for padd word

In [14]:
tag_2_index = {t:i for i,t in enumerate(tags)}
y = [[tag_2_index[w[1]] for w in s] for s in sentences]
y_pad = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag_2_index["O"])
y_pad[1]

array([13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 16,  0,
        0,  0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0], dtype=int32)

In [26]:
batch_size = 32
l = len(x_pad) // batch_size // 25 * 25 * batch_size 
X_train, X_test, Y_train, Y_test = train_test_split(x_pad[:l], y_pad[:l], test_size=0.2)
Y_train = Y_train.reshape(Y_train.shape[0], Y_train.shape[1], 1)

print(*map(len, [X_train, X_test, Y_train, Y_test]))

37760 9440 37760 9440


## Loading gazetteers

In [16]:
os.listdir('/content/gdrive/My Drive/gazetteers')

['caprovinces.txt',
 'countries.txt',
 'nationalities.txt',
 'uscities.txt',
 'mexstates.txt',
 'isocountries.txt',
 'usstates.txt',
 'usstateabbrev.txt']

In [17]:
gazetteers = {}
for gz in os.listdir('/content/gdrive/My Drive/gazetteers'):
    i = set(open(f'/content/gdrive/My Drive/gazetteers/{gz}', 'r', encoding="latin1").read().strip().split('\n'))
    gazetteers[gz.split('.')[0]] = i

In [18]:
list(gazetteers['countries'])[:10]

['Antigua and Barbuda',
 'Belarus',
 'Herzegovina',
 'Netherlands Antilles',
 'Sierra Leone',
 'Aland',
 'Ireland',
 'Liberia',
 'Gabon',
 'Trinidad']

# Сreating model

ELMo embedder for word embbeding 

In [19]:
elmo_model = hub.load('https://tfhub.dev/google/elmo/2').signatures['tokens']
def elmo_emb(x):
    return elmo_model(tokens=tf.squeeze(tf.cast(x, tf.string)), 
                      sequence_len=tf.constant(batch_size*[max_len]))["elmo"]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Function to extract gazzeteer feature for each word

In [20]:
#binary feature: 1 if word in gazetteers else 0
def GazetteerFeatures(sentences):
    result = []
    for s in sentences:
        data = []
        for w in s:
            temp = []
            for g in gazetteers.values():
                temp.append(1 if w in g else 0)

            data.append(temp)
        result.append(data)
                
    return np.array(result, dtype=np.float32)

Model definition

In [28]:
    input_txt = Input(shape=(max_len,), dtype=tf.string)
    input_gaz  = Input(shape=(max_len, 8), dtype=tf.float32)
    elmo_embedding = Lambda(elmo_emb, output_shape=(max_len, 1024))(input_txt)
    emb = Concatenate()([elmo_embedding, input_gaz])
    x = Bidirectional(LSTM(units=512, return_sequences=True,recurrent_dropout=0.1, dropout=0.1))(emb)
    out = TimeDistributed(Dense(17, activation="softmax"))(x)
    model = Model([input_txt, input_gaz], out)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 104)          0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 104, 1024)    0           input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 104, 8)       0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 104, 1032)    0           lambda_2[0][0]                   
                                                                 input_4[0][0]              

## Train and Test section

In [22]:
history = model.fit([np.array(X_train), GazetteerFeatures(X_train)], Y_train, validation_split = 0.2,
                    batch_size=batch_size, epochs=1, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 30400 samples, validate on 7600 samples
Epoch 1/1
30400/30400 [==============================] - 553s 18ms/step - loss: 0.0558 - accuracy: 0.9851 - val_loss: 0.0256 - val_accuracy: 0.9922


In [23]:
!pip install seqeval

In [29]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
model_predict = model.predict([np.array(X_test), GazetteerFeatures(X_test)], verbose=1)

9440/9440 [==============================] - 154s 16ms/step


TODO:DEL
### Result and Score section

In [30]:
idx2tag = {i: w for w, i in tag_2_index.items()}

def pred_2_label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace(padding, "O"))
        out.append(out_i)
    return out

def test_2_label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p].replace(padding, "O"))
        out.append(out_i)
    return out
    
predicted_labels = pred_2_label(model_predict)
test_labels = test_2_label(Y_test)
print(classification_report(test_labels, predicted_labels))


           precision    recall  f1-score   support

      geo       0.01      0.01      0.01      7370
      per       0.00      0.01      0.01      3346
      tim       0.00      0.03      0.01      4048
      org       0.00      0.01      0.00      4011
      gpe       0.04      0.15      0.07      3109
      eve       0.00      0.00      0.00        76
      nat       0.00      0.23      0.00        53
      art       0.00      0.06      0.00        71

micro avg       0.00      0.03      0.01     22084
macro avg       0.01      0.03      0.02     22084



In [31]:
i = 375
p = model.predict([np.array(X_test[i:i+batch_size]), GazetteerFeatures(X_test[i:i+batch_size])])[0]
p = np.argmax(p, axis=-1)
print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
print("="*30)
for w, true, pred in zip(X_test[i], Y_test[i], p):
    if w != padding:
        print("{:15}:{:5} ({})".format(w, tags[pred], tags[true]))

Word            Pred : (True)
North          :B-art (B-geo)
Korea          :B-art (I-geo)
says           :B-gpe (O)
it             :B-gpe (O)
will           :B-gpe (O)
delay          :B-gpe (O)
a              :B-gpe (O)
scheduled      :B-gpe (O)
round          :B-gpe (O)
of             :B-gpe (O)
high           :B-gpe (O)
level          :B-gpe (O)
inter-Korean   :B-gpe (O)
talks          :B-gpe (O)
,              :B-gpe (O)
because        :B-gpe (O)
of             :B-gpe (O)
an             :B-gpe (O)
upcoming       :B-gpe (O)
joint          :B-gpe (O)
military       :B-gpe (O)
exercise       :I-org (O)
between        :B-geo (O)
South          :I-art (B-geo)
Korea          :B-gpe (I-geo)
and            :B-gpe (O)
the            :B-gpe (O)
United         :I-org (B-geo)
States         :I-org (I-geo)
.              :I-art (O)


## References

https://www.kaggle.com/abhinavwalia95entity-annotated-corpus/home#ner.csv

https://www.kaggle.com/nltkdata/gazetteers

https://arxiv.org/pdf/1802.05365.pdf

https://arxiv.org/pdf/1603.01354.pdf

https://arxiv.org/pdf/1901.08746.pdf

https://www.aclweb.org/anthology/W19-5807.pdf

https://programminghistorian.org/en/lessons/extracting-keywords

https://towardsdatascience.com/named-entity-recognition-ner-meeting-industrys-requirement-by-applying-state-of-the-art-deep-698d2b3b4ede

https://www.kaggle.com/nltkdata/conll-corpora

https://mc.ai/how-to-build-deep-neural-network-for-custom-ner-with-keras/

https://medium.com/analytics-vidhya/elmo-embedding-the-entire-intent-of-a-query-530b268c4cd

https://www.depends-on-the-definition.com/named-entity-recognition-conditional-random-fields-python/

https://towardsdatascience.com/elmo-embeddings-in-keras-with-tensorflow-hub-7eb6f0145440

https://confusedcoders.com/data-science/deep-learning/how-to-build-deep-neural-network-for-custom-ner-with-keras